In [1]:
# 讀檔
import csv
with open('movies.csv', encoding = 'utf8') as csvfile:
    # 解析內容
    rows = list(csv.reader(csvfile))

In [2]:
#分配測試資料與訓練資料
test_movie = []
train_movie = []
for movie in rows[1:]:
    if movie[2] != 'NA':
        if len(test_movie) < 500:
            test_movie.append(movie)
        else:
            train_movie.append(movie)

#只取第一個分類
for movie in test_movie:
    temp = movie[2].split(",")
    movie[2] = temp[0]

for movie in train_movie:
    temp = movie[2].split(",")
    movie[2] = temp[0]

In [3]:
#有哪些分類
categories = []

for movie in train_movie:
    temp = movie[2].split(",")
    categories.append(temp[0])

categories = set(categories)
categories = list(categories)
print(categories)

['犯罪', '勵志', '溫馨/家庭', '愛情', '紀錄片', '音樂/歌舞', '科幻', '懸疑/驚悚', '奇幻', '喜劇', '影展', '歷史/傳記', '武俠', '劇情', '戰爭', '動作', '冒險', '恐怖', '戲劇', '動畫']


In [4]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import math
from sklearn.preprocessing import normalize

In [5]:
#資料分詞
vectorizer = TfidfVectorizer(sublinear_tf=False, stop_words=None, token_pattern="(?u)\\b\\w+\\b", smooth_idf=True, norm='l2')
data = []
for i in test_movie:
    data.append(i[3][-150:])
for i in train_movie:
    data.append(i[3][-150:])
tfidf = vectorizer.fit_transform(data)
tfidf = tfidf.toarray()
#df_tfidf = pd.DataFrame(tfidf.toarray(),columns=vectorizer.get_feature_names(), index=range(len(train_data))).transpose()

#df_tfidf
#print("TFIDF")

In [6]:
df_tfidf = pd.DataFrame(tfidf,columns=vectorizer.get_feature_names(), index=range(len(data)))
df_tfidf

,0,00,000,000個家庭出租社會住宅,000公里的,000冊,000名以上索馬利亞暴民的猛烈攻擊,000字,000歐元全數送給她作為一個月的薪水,000萬冊,...,李承鉉,理解,０,１,２,２人同行優惠價４００元,２月更是有4場總計10部話題短片席捲而來,ｊ幫,ｊｋ羅琳,ｋｎｂ特效菁英製作公司來處理特殊效果的部份
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#訓練資料 分類決定y值
y_train = []
for movie in train_movie:
    for i in range(len(categories)):
        if movie[2] == categories[i]:
            y_train.append(i)

In [10]:
#訓練資料 x值為tfidf
x_train = list(tfidf[500:])

In [11]:
#測試資料 x值為tfidf值
x_test = list(tfidf[:500])

In [12]:
from sklearn.neighbors import KNeighborsClassifier

#knn k值=3開始測試
knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(x_train,y_train)
pred = knn.predict(x_test)

In [13]:
#預測結果
y_test = []

for i in pred:
    y_test.append(categories[i])

In [20]:
#準確率
correct = 0
for i in range(500):
    if y_test[i] == test_movie[i][2]:
        correct = correct + 1

accuracy = correct / 500
print(accuracy)

0.262


In [21]:
#預測情形
for i in range(500):
    print(test_movie[i][1])
    print('fact: ', test_movie[i][2])
    print('pred: ', y_test[i])
    print('----------------------------------')

一世狂野
fact:  劇情
pred:  動作
----------------------------------
玩命關頭
fact:  動作
pred:  動作
----------------------------------
戰雲密佈
fact:  動作
pred:  動作
----------------------------------
騎士風雲錄
fact:  動作
pred:  動作
----------------------------------
金法尤物
fact:  喜劇
pred:  動作
----------------------------------
瘋狂世界
fact:  冒險
pred:  動作
----------------------------------
震撼教育
fact:  劇情
pred:  劇情
----------------------------------
神鬼第六感
fact:  劇情
pred:  劇情
----------------------------------
觸不到的戀人
fact:  奇幻
pred:  動作
----------------------------------
北京樂與路
fact:  劇情
pred:  劇情
----------------------------------
晚孃
fact:  劇情
pred:  劇情
----------------------------------
人間有情天
fact:  劇情
pred:  恐怖
----------------------------------
美國派2
fact:  喜劇
pred:  動作
----------------------------------
我和吸血鬼有份合約
fact:  劇情
pred:  懸疑/驚悚
----------------------------------
千禧曼波
fact:  劇情
pred:  劇情
----------------------------------
救世主
fact:  動作
pred:  動作
----------------------------------
鬼計神偷
fact:  劇情
pred:  劇情
----

pred:  動作
----------------------------------
雙面驚悚
fact:  犯罪
pred:  動作
----------------------------------
七夜怪談西洋篇
fact:  劇情
pred:  動作
----------------------------------
不速之客
fact:  劇情
pred:  動作
----------------------------------
巔峰殺戮
fact:  動作
pred:  動作
----------------------------------
攔截密碼戰
fact:  劇情
pred:  懸疑/驚悚
----------------------------------
金牌間諜
fact:  動作
pred:  動作
----------------------------------
魔戒二部曲：雙城奇謀.
fact:  動作
pred:  動作
----------------------------------
無間道
fact:  劇情
pred:  動作
----------------------------------
三更
fact:  劇情
pred:  動作
----------------------------------
神鬼交鋒
fact:  劇情
pred:  動作
----------------------------------
完美女人
fact:  科幻
pred:  動作
----------------------------------
異度空間
fact:  恐怖
pred:  動作
----------------------------------
英雄
fact:  動作
pred:  動作
----------------------------------
貼身情人
fact:  喜劇
pred:  懸疑/驚悚
----------------------------------
木偶奇遇記
fact:  奇幻
pred:  動作
----------------------------------
老大靠邊閃：歪打正著
fact:  犯罪
pred:  劇情
-----------